
## Rapidly exploring Random Tree (RRT)

In [1]:
mapWidth = 100;
mapHeight = 100;

obstacles = [
    20, 20, 10, 20;
    50, 50, 15, 10;
    30, 70, 20, 10;
    60, 80, 10, 10;
];

start = [10, 10];
goal = [90, 90];


Setting up map dimensions, starting point, ending point(Goal) and obstacles.


In [2]:
figure;
hold on;
axis([0 mapWidth 0 mapHeight]);
title('Map with Obstacles');
xlabel('X');
ylabel('Y');
grid on;
axis equal;

for i = 1:size(obstacles, 1)
    rectangle('Position', obstacles(i, :), 'FaceColor', [0.5 0.5 0.5]);
end

plot(start(1), start(2), 'go', 'MarkerSize', 10, 'LineWidth', 2);
plot(goal(1), goal(2), 'ro', 'MarkerSize', 10, 'LineWidth', 2);


Map visulation including obstacles


In [3]:
rrt_path_planner(start, goal, obstacles, mapWidth, mapHeight);

function rrt_path_planner(start, goal, obstacles, mapWidth, mapHeight)
    max_iter = 500;
    step_size = 5;
    goal_threshold = 5;

    tree.nodes = start;
    tree.parents = 0;


-  **max\_iter**: Maximum number of iterations for the RRT algorithm.
-  **step\_size**: The maximum distance the tree can extend towards a random point in each iteration.
-  **goal\_threshold**: The distance within which the goal is considered reached.
-  **tree**: A structure that stores the nodes of the RRT and their parent nodes. It starts with the initial node (start position).

In [4]:
for iter = 1:max_iter
    rand_point = [rand()*mapWidth, rand()*mapHeight];
    [nearest_node, nearest_index] = find_nearest_node(tree, rand_point);
    new_node = move_towards(nearest_node, rand_point, step_size);

    if ~check_collision(nearest_node, new_node, obstacles)
        tree.nodes = [tree.nodes; new_node];
        tree.parents = [tree.parents; nearest_index];
        plot([nearest_node(1), new_node(1)], [nearest_node(2), new_node(2)], 'b-');
        drawnow;

        if norm(new_node - goal) < goal_threshold
            % Trace the path back to the start
            path = [new_node];
            while nearest_index ~= 0
                path = [tree.nodes(nearest_index, :); path];
                nearest_index = tree.parents(nearest_index);
            end
            path = [start; path];
            plot(path(:,1), path(:,2), 'r-', 'LineWidth', 2);
            disp('Goal reached!');
            hold off;
            return;
        end
    end
end
disp('Goal not reached within the maximum iterations.');
hold off;


-  **rand\_point**: A random point generated within the map bounds.
-  **find\_nearest\_node**: Finds the node in the tree that is closest to the random point.
-  **move\_towards**: Creates a new node by moving from the nearest node towards the random point, constrained by the step size.
-  **check\_collision**: Ensures that the new node doesn’t collide with any obstacles.
-  **tree**: Updated with the new node if it’s valid (no collision).
-  **plot**: Visualizes the path as the tree grows.
-  **goal check**: If the new node is close enough to the goal, the path is traced back to the start and displayed.

In [5]:
function [nearest_node, nearest_index] = find_nearest_node(tree, point)
    distances = vecnorm(tree.nodes - point, 2, 2);
    [~, nearest_index] = min(distances);
    nearest_node = tree.nodes(nearest_index, :);
end


-  Calculates the Euclidean distance from each node in the tree to the random point.
-  Returns the closest node (<samp>nearest_node</samp>) and its index in the tree (<samp>nearest_index</samp>).

In [6]:
function new_node = move_towards(nearest_node, rand_point, step_size)
    direction = rand_point - nearest_node;
    direction = direction / norm(direction);
    new_node = nearest_node + step_size * direction;
end


-  **direction**: A unit vector pointing from the nearest node to the random point.
-  **new\_node**: A new node generated by moving from the nearest node towards the random point by a distance defined by <samp>step_size</samp>.

In [7]:
function collision = check_collision(node1, node2, obstacles)
    collision = false;
    for i = 1:size(obstacles, 1)
        if line_rect_intersection(node1, node2, obstacles(i, :))
            collision = true;
            break;
        end
    end
end


-  Checks if the line segment between two nodes intersects with any of the obstacles.
-  **line\_rect\_intersection**: Determines whether a line segment intersects with a given rectangle (obstacle).

In [8]:
function intersect = line_rect_intersection(p1, p2, rect)
    x1 = rect(1); y1 = rect(2); x2 = rect(1)+rect(3); y2 = rect(2)+rect(4);
    if (line_segment_intersect(p1, p2, [x1 y1], [x2 y1]) || ...
        line_segment_intersect(p1, p2, [x2 y1], [x2 y2]) || ...
        line_segment_intersect(p1, p2, [x2 y2], [x1 y2]) || ...
        line_segment_intersect(p1, p2, [x1 y2], [x1 y1]))
        intersect = true;
    end
end


-  This function checks if the line segment between two points (<samp>p1</samp> and <samp>p2</samp>) intersects with any side of the rectangle (<samp>rect</samp>).
-  It checks for intersections with all four sides of the rectangle.

In [9]:
function intersect = line_segment_intersect(p1, p2, q1, q2)
    a1 = p2 - p1; a2 = q2 - q1;
    cross_prod = a1(1) * a2(2) - a1(2) * a2(1);
    if abs(cross_prod) < 1e-10
        return;
    end
    t = ((q1(1) - p1(1)) * a2(2) - (q1(2) - p1(2)) * a2(1)) / cross_prod;
    u = ((q1(1) - p1(1)) * a1(2) - (q1(2) - p1(2)) * a1(1)) / cross_prod;
    if t >= 0 && t <= 1 && u >= 0 && u <= 1
        intersect = true;
    end
end


-  Determines if two line segments intersect.
-  Uses vector cross products to check for intersection within the bounds of the segments.
